In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

def preprocess_data(data):
    label_encoder = LabelEncoder()
    data['label'] = label_encoder.fit_transform(data['label'])

    data = pd.get_dummies(data, columns=['src', 'dst', 'Protocol'])

    #print(combined_data.isnull().sum())
    data = data.dropna()

    X = data.drop('label', axis=1)
    y = data['label']
    return X,y

def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def load_model(model_filename):
    loaded_model = joblib.load(model_filename)
    return loaded_model

loaded_model = load_model('/content/ddos_model.pkl')
data = load_data("/content/testdata.csv")
X,y= preprocess_data(data)
y_pred=loaded_model.predict(X)
accuracy=accuracy_score(y,y_pred)
